In [227]:
import requests
import os
import time
import json
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession

In [57]:
os.makedirs('data/county')

In [250]:
def create_file(response, name, ext, mode):
    
    county_path = 'data/county/'
    
    file = open('{}{}.{}'.format(county_path, name, ext), mode)
    file.write(response)
    file.close()

In [261]:
url_ca = 'https://data.chhs.ca.gov/dataset/f333528b-4d38-4814-bebb-12db1f10f535/resource/046cdd2b-31e5-4d34-9ed3-b48cdbc4be7a/download/covid19cases_test.csv'
response_ca = requests.get(url_ca)

create_file(response_ca.text, 'california', 'csv', 'x')

In [262]:
url_tx = 'https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyCaseCountData.xlsx'
response_tx = requests.get(url_tx)

create_file(response_tx.content, 'texas', 'xlsx', 'wb')

In [267]:
offset = 0
limit = 2000

In [264]:
for i in range(5): #while True:

    url_fl = 'https://services1.arcgis.com/CY1LXxl9zlJeBuRZ/arcgis/rest/services/Florida_COVID19_Case_Line_Data_NEW/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&resultOffset={}&resultRecordCount={}&f=json'.format(offset, limit)
    response_fl = requests.get(url_fl)
    data_fl = []
    
    if response_fl.text.strip() != '[]':
        
        current_response_data = [feature['attributes'] for feature in response_fl.json()['features']]
        
        for row in current_response_data:
            data_fl.append(row)
    else:
        break
    
    offset += limit
    time.sleep(1)
    
with open('data/county/florida.json', 'w') as florida_file:
    json.dump(data_fl, florida_file)

In [268]:
for i in range(5): #while True:

    url_ny = 'https://health.data.ny.gov/resource/xdss-u53e.json?$limit={}&$offset={}'.format(limit, offset)
    response_ny = requests.get(url_ny)
    data_ny = []
    
    if response_ny.text.strip() != '[]':
        for row in response_ny.json():
            data_ny.append(row)
    else:
        break
    
    offset += limit
    time.sleep(5)
    
with open('data/county/new_york.json', 'w') as ny_file:
    json.dump(data_ny, ny_file)

In [269]:
for i in range(5): #while True:

    url_pa = 'https://data.pa.gov/resource/j72v-r42c.json?$limit={}&$offset={}'.format(limit, offset)
    response_pa = requests.get(url_pa)
    data_pa = []
    
    if response_pa.text.strip() != '[]':
        for row in response_pa.json():
            data_pa.append(row)
    else:
        break
    
    offset += limit
    time.sleep(5)
    
with open('data/county/pennsylvania.json', 'w') as penn_file:
    json.dump(data_pa, penn_file)

In [270]:
url_counties_il = 'https://en.wikipedia.org/wiki/List_of_counties_in_Illinois'
response_url_counties_il = requests.get(url_counties_il)
soup = BeautifulSoup(response_url_counties_il.text, 'html.parser')

In [271]:
counties_il = []
for county in soup.select('tbody > tr > th > a'):
    counties_il.append(county.get_text())

In [272]:
counties_il = [county.split(' ')[0] for county in counties_il]

In [274]:
data_il = []

for county in counties_il[:3]:
    url_il = 'https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetCountyHistorical?countyName={}'.format(county)
    response_il = requests.get(url_il)
    
    for row in response_il.json()['values']:
        data_il.append(row)
        
    time.sleep(5)
        
with open('data/county/illinois.json', 'w') as illinois_file:
    json.dump(data_il, illinois_file)  

In [275]:
url_oh = 'https://coronavirus.ohio.gov/static/dashboards/COVIDDeathData_CountyOfResidence.csv'
response_oh = requests.get(url_oh)

create_file(response_oh.text, 'ohio', 'csv', 'x')

In [260]:
url_ga = 'https://opendata.arcgis.com/datasets/d817e06e4b264905a075b9332cd41962_0.geojson'
response_ga = requests.get(url_ga)
data_ga = []

for row in response_ga.json()['features']:
    data_ga.append(row['properties'])

with open('data/county/georgia.json', 'w') as georgia_file:
    json.dump(data_ga, georgia_file)

In [276]:
url_usafacts_cases = 'https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv?_ga=2.249922758.1108579064.1615472677-188720245.1615472677'
response_cases = requests.get(url_usafacts_cases)

create_file(response_cases.text, 'cases', 'csv', 'x')

In [277]:
url_usafacts_deaths = 'https://static.usafacts.org/public/data/covid-19/covid_deaths_usafacts.csv?_ga=2.247303236.1108579064.1615472677-188720245.1615472677'
response_deaths = requests.get(url_usafacts_deaths)

create_file(response_deaths.text, 'deaths', 'csv', 'x')

In [278]:
url_usafacts_population = 'https://static.usafacts.org/public/data/covid-19/covid_county_population_usafacts.csv?_ga=2.246842820.1108579064.1615472677-188720245.1615472677'
response_population = requests.get(url_usafacts_population)

create_file(response_population.text, 'population', 'csv', 'x')